# Track military vehicles in video inputs

In this notebook, we'll use the model that we trained with our custom dataset to track military vehicles in videos. We'll use the [yolo-tracking](https://github.com/mikel-brostrom/yolo_tracking) library to provide state-of-the-art multi-object tracking algorithms.

### Setup

Clone the [yolo-tracking](https://github.com/mikel-brostrom/yolo_tracking) repository and check PyTorch and GPU support.

In [ ]:
!git clone https://github.com/mikel-brostrom/yolo_tracking.git  # clone repo
%cd yolo_tracking
!pip install -v -e .
%pip install ultralytics lap
%cd ..

import torch
from IPython.display import Image, clear_output

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

### Download model and video samples

We'll download the model weights and video samples from the project's repository

In [1]:
# get the test video from the repo
!wget -nc https://github.com/jonasrenault/adomvi/raw/main/resources/test/tank2.mp4

# get the model weights
!wget https://github.com/jonasrenault/adomvi/releases/download/v1.0.0/yolov8n_tanks.zip

--2023-06-12 15:18:46--  https://github.com/jonasrenault/adomvi/raw/main/resources/tank2.mp4
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/jonasrenault/adomvi/main/resources/tank2.mp4 [following]
--2023-06-12 15:18:46--  https://raw.githubusercontent.com/jonasrenault/adomvi/main/resources/tank2.mp4
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3540328 (3,4M) [application/octet-stream]
Saving to: ‘tank2.mp4’

tank2.mp4           100%[===================>]   3,38M  19,4MB/s    in 0,2s    

2023-06-12 15:18:46 (19,4 MB/s) - ‘tank2.mp4’ saved [3540328/3540328]

--2023-06-12 15:18:47--  https://g

Unzip the model weights

In [3]:
!unzip yolov8n_tanks.zip; rm yolov8n_tanks.zip
!mv runs/detect/train/weights/best.pt yolov8n-tanks.pt

### Run inference on the video

We run our detection model on the sample video, saving the output to disk.

In [6]:
!python yolo_tracking/examples/track.py --yolo-model yolov8n-tanks.pt --reid-model mobilenetv2_x1_4_dukemtmcreid.pt --source tank2.mp4 --save --project runs/track

2023-06-12 15:31:18.108 | INFO     | boxmot.utils.torch_utils:select_device:52 - Yolo Tracking v10.0.13 🚀 Python-3.10.10 torch-2.0.1 CPU

2023-06-12 15:31:18.108 | INFO     | __main__:run:66 - {'yolo_model': PosixPath('yolov8n-tanks.pt'), 'reid_model': PosixPath('/Users/jonas/workspace/yolo_tracking/examples/weights/mobilenetv2_x1_4_dukemtmcreid.pt'), 'tracking_method': 'deepocsort', 'source': 'tank2.mp4', 'imgsz': [640], 'conf': 0.5, 'iou': 0.7, 'device': '', 'show': False, 'save': True, 'classes': None, 'project': 'runs/track', 'name': 'exp', 'exist_ok': False, 'half': False, 'vid_stride': 1, 'hide_label': False, 'hide_conf': False, 'save_txt': False}
2023-06-12 15:31:18.396 | SUCCESS  | boxmot.deep.reid_model_factory:load_pretrained_weights:229 - Successfully loaded pretrained weights from "/Users/jonas/workspace/yolo_tracking/examples/weights/mobilenetv2_x1_4_dukemtmcreid.pt"
2023-06-12 15:31:18.396 | WARNING  | boxmot.deep.reid_model_factory:load_pretrained_weights:233 - The follo

### Display the result video

Compress the video file to a format supported by Google Colab (mpeg4 (native) -> h264 (libx264))

In [17]:
!ffmpeg -i runs/track/exp/tank2.mp4 -vf fps=30 -vcodec libx264 output.mp4

ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.3 (clang-1403.0.22.14.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --di

Display it with HTML

In [18]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('output.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

In [19]:
HTML("""
<video controls width="640">
      <source src="%s" type="video/mp4">
</video>
""" % data_url)